<a href="https://colab.research.google.com/github/bhowad-akash/Machine-Learning/blob/main/Large_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222321 sha256=25a72792a8622e2769457bd39acf5fd9c8ca71bbcbf87fe07100b8a18aa35361
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


In [17]:
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pylzma
import pandas as pd
import torch


'''CPU are sequential in nature, processing the data one aftre the other.
Whereas, GPUs can process the multiple data parallely.'''

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [3]:
!python --version

Python 3.10.12


In [5]:
from google.colab import files
uploaded = files.upload()

Saving wizard_of_oz.txt to wizard_of_oz.txt


In [7]:
import os
os.listdir()

['.config', 'wizard_of_oz.txt', 'sample_data']

In [8]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [9]:
print(text[:200])

﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [10]:
#Printing all the characters in the text piece
chars = sorted(set(text))
print(chars, len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff'] 81


In [11]:
#Building a Tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

print(string_to_int)
print(int_to_string)

'''Encode is a lambda function that takes a string s as input and returns integer values from the 'string_to_int' dictionary,
whereas decode is a lambda function that takes integers as a input and returns characters, with join forming the resulting string'''

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31:

In [12]:
data = torch.tensor(encode(text), dtype=torch.long)    #encode all the words present in the text piece
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [13]:
#Dividing the data into Training and Validation
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]
print(len(data), len(train_data), len(valid_data))

232309 185847 46462


In [16]:
#Function to generate batches of data for training or validation
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) #Randomly select starting indices for the batch
  x = torch.stack([data[i:i+block_size] for i in ix])       #Create input sequences by stacking slices of data starting at each index in 'ix'
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])   #Create input sequences by stacking slices of data starting at each index in 'ix'
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[62, 75, 58, 57,  9,  1, 33,  1],
        [57,  1, 65, 68, 75, 62, 67, 60],
        [ 0,  0, 44, 61, 58,  1, 47, 62],
        [68, 66, 68, 71, 71, 68, 76,  9]], device='cuda:0')
targets:
tensor([[75, 58, 57,  9,  1, 33,  1, 56],
        [ 1, 65, 68, 75, 62, 67, 60, 65],
        [ 0, 44, 61, 58,  1, 47, 62, 79],
        [66, 68, 71, 71, 68, 76,  9,  3]], device='cuda:0')
